In [105]:
import pandas as pd
from glob import glob
import numpy as np
from easydict import EasyDict as edict
import os
import time
import sys
from tabulate import tabulate

sys.path.insert(0,os.path.expanduser('~/git/torchseg'))
print(sys.path)

from utils.config import load_config
from utils.summary_to_csv import config_to_log,load_log,edict_to_pandas,today_log,recent_log

def summary(rootpath,tags,filter_str=None,):
    config_files=glob(os.path.join(rootpath,'**','config.txt'),recursive=True)
    if filter_str == 'today':
        config_files=today_log(config_files)
    elif filter_str == 'recent':
        config_files=recent_log(config_files)
    elif filter_str is not None:
        config_files=[f for f in config_files if f.find(filter_str)>=0]
    
    tasks=pd.DataFrame()
    for cfg in config_files:
        log=config_to_log(cfg)
        if log is not None:
            ed=load_config(cfg)
            metrics=load_log(log,tags)
            for key,value in metrics.items():
                ed[key]=value
            
            ed['dir']=cfg
            log_time=''
            for s in cfg.split(os.sep):
                if s.find('___')>=0:
                    log_time=s
                    break
            ed['log_time']=log_time
            
            task=edict_to_pandas(ed)
            tasks=tasks.append(task,ignore_index=True,sort=False)
        else:
            print('cannot find log file for',cfg)
            
    return tasks

def dump(tags=['train/fmeasure','val/fmeasure'],
        rootpath=os.path.expanduser('~/tmp/logs/motion'),
        notes=['motion_fcn','motion_sparse','motion_psp'],
        note_gtags=None,
        descripe=['note','epoch'],
        invalid_param_list=['dir','log_time','root_path',
                            'test_path', 'train_path', 'val_path'],
        delete_nan=False,
        dump_group=False,
        dump_dir=True):
    dir_tags=[tags[1],'dir']
    
    for idx,note in enumerate(notes):
        show_tags=[]
        show_tags=tags.copy()
        tasks=summary(rootpath,tags,note)
        if tasks.empty:
            print(note,ntag,'task is empty')
            continue
        
        if note_gtags is None:
            param_list=[]
            for col in tasks.columns:
                if not isinstance(tasks[col][0],(tuple,list)):
                    if len(set(tasks[col]))>1 and col not in invalid_param_list and col not in tags:
                        param_list.append(col)

            print(note,param_list)
            group_tags=param_list
        else:
            if idx<len(note_gtags):
                group_tags=note_gtags[idx]
            else:
                group_tags=[]
            
        show_tags+=group_tags
        show_tags+=descripe
        show_tags=list(set(show_tags))
        
        # remove empty dirs
        if delete_nan:
            dirs=tasks[tasks[tags[1]].isna()]['dir'].tolist()
            tasks=tasks[tasks[tags[1]].notna()]
            for dir in dirs:
                print(os.path.dirname(dir))
                os.system('rm -rf {}'.format(os.path.dirname(dir)))

        #print(tasks[show_tags].groupby(group_tags).max().to_string())
        if dump_group:
            print(tabulate(tasks[show_tags].groupby(group_tags).mean().sort_values(tags[1]),tablefmt='pipe',headers='keys'))
            print('\n')
            print(tabulate(tasks[[tags[1]]+group_tags].groupby(group_tags).agg([np.mean,np.std,np.max]),tablefmt='pipe',headers='keys'))
            print('\n')
        
        if note=='recent':
            print(tabulate(tasks[show_tags].sort_values('log_time'),tablefmt='pipe',headers='keys'))
        else:
            print(tabulate(tasks[show_tags].sort_values(tags[1]),tablefmt='pipe',headers='keys'))
        print('\n')
        if dump_dir:
            print(tabulate(tasks[dir_tags].sort_values(tags[1]),tablefmt='pipe',headers='keys'))
            print('\n')

['/home/yzbx/git/torchseg', '/home/yzbx/git/torchseg', '/home/yzbx/git/torchseg', '/home/yzbx/git/torchseg', '/home/yzbx/git/torchseg', '/home/yzbx/git/torchseg', '/home/yzbx/git/torchseg', '/home/yzbx/git/torchseg', '/home/yzbx/git/torchseg', '/home/yzbx/git/torchseg', '/home/yzbx/git/torchseg', '/home/yzbx/git/torchseg', '/home/yzbx/git/torchseg', '/home/yzbx/git/torchseg', '', '/home/yzbx/git/torchseg/notebooks', '/home/yzbx/git/gnu/models/research', '/home/yzbx/git/gnu/models/research/slim', '/home/yzbx/bin/miniconda3/envs/new/lib/python36.zip', '/home/yzbx/bin/miniconda3/envs/new/lib/python3.6', '/home/yzbx/bin/miniconda3/envs/new/lib/python3.6/lib-dynload', '/home/yzbx/bin/miniconda3/envs/new/lib/python3.6/site-packages', '/home/yzbx/bin/miniconda3/envs/new/lib/python3.6/site-packages/cityscapesscripts-1.0.0-py3.6-linux-x86_64.egg', '/home/yzbx/bin/miniconda3/envs/new/lib/python3.6/site-packages/torchvision-0.2.1-py3.6.egg', '/home/yzbx/bin/miniconda3/envs/new/lib/python3.6/site-

In [106]:
tags=['train/fmeasure','val/fmeasure']
rootpath=os.path.expanduser('~/tmp/logs/motion')
note_dict={
    'task063':['dataset','decode_main_layer'],
    'task065':['dataset','decode_main_layer'],
    'task066':['freeze_layer','decode_main_layer'],
}
notes=[key for key in note_dict.keys()]
dump(tags=tags,rootpath=rootpath,notes=notes,descripe=['note','epoch'],delete_nan=False,dump_group=True)

task063 ['dataset', 'decode_main_layer']
|                  |   train/fmeasure |   val/fmeasure |   epoch |
|:-----------------|-----------------:|---------------:|--------:|
| ('cdnet2014', 1) |         0.963229 |       0.6464   |      30 |
| ('cdnet2014', 0) |         0.957614 |       0.653154 |      30 |
| ('FBMS', 1)      |         0.957224 |       0.798691 |      30 |
| ('FBMS', 0)      |         0.976752 |       0.799106 |      30 |


|                  |   ('val/fmeasure', 'mean') |   ('val/fmeasure', 'std') |   ('val/fmeasure', 'amax') |
|:-----------------|---------------------------:|--------------------------:|---------------------------:|
| ('FBMS', 0)      |                   0.799106 |                       nan |                   0.799106 |
| ('FBMS', 1)      |                   0.798691 |                       nan |                   0.798691 |
| ('cdnet2014', 0) |                   0.653154 |                       nan |                   0.653154 |
| ('cdnet2014', 1) |

In [107]:
tags=['train/fmeasure','val/fmeasure']
rootpath=os.path.expanduser('~/tmp/logs/motion')
notes=['today','recent']
note_gtags=[['log_time','net_name'],
           ['log_time','net_name']]
dump(tags=tags,rootpath=rootpath,notes=notes,note_gtags=note_gtags,descripe=['note','epoch'],delete_nan=False,dump_group=False)

|    | note    | net_name          |   train/fmeasure |   val/fmeasure |   epoch | log_time              |
|---:|:--------|:------------------|-----------------:|---------------:|--------:|:----------------------|
|  5 | test    | motion_panet2_stn |         0.122733 |       0.176008 |       3 | 2019-04-10___16-05-44 |
|  4 | test    | motion_panet2     |         0.371574 |       0.190923 |       3 | 2019-04-10___14-17-36 |
|  2 | task066 | motion_panet2     |         0.950078 |       0.665045 |      30 | 2019-04-10___00-44-47 |
|  1 | task066 | motion_panet2     |         0.954774 |       0.67707  |      30 | 2019-04-10___00-09-12 |
|  3 | task066 | motion_panet2     |         0.950328 |       0.700768 |      30 | 2019-04-10___01-10-00 |
|  0 | task065 | motion_panet2     |         0.950146 |       0.713523 |      30 | 2019-04-10___00-08-59 |


|    |   val/fmeasure | dir                                                                                         |
|---:|---------------:|:

['batch_size', 'net_name', 'note', 'use_optical_flow', 'train/fmeasure', 'val/fmeasure', 'dir', 'log_time']
<class 'str'>
